In [1]:
import statsmodels.api as sm


def smooth(y):
    #return savitzky_golay(y, window_size=2001, order=3)
    return lowess(y)

# 0.2가 제일 잘 없앴음
def lowess(y, f=0.2):
    x = np.arange(0, len(y))
    return sm.nonparametric.lowess(y, x, frac=f, it=0)[:, 1].T


import numpy as np
import pandas as pd
import os, pickle, sys
from scipy import signal


### input 설정
SRATE = 250 # 250Hz
LEN_INPUT = 20 # input 10s
LEN_PER_NRS = 60 # vital length for each NRS
OVERLAP = 2
n_aug = int((LEN_PER_NRS-LEN_INPUT)/OVERLAP) + 1 # data augmentation 개수

non_preop_list = []


input_path = '../DL_model/dataset/preprocess4/input3/'
if not os.path.exists(input_path[:-1]):
    os.mkdir(input_path[:-1])


if True:
    false_row_list_preop = []
    
    preop_ecg_path = "../../cranberry2/Preprocessing/vital_data/ECG_250Hz_preop_2min/"
    vital_path = '../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_preop_2min/'
    df_preprocess_preop = pickle.load(open('../DL_model/cache/preprocess3/input3/df_preprocess_preop_agender','rb'))
    df_demograph = pd.read_csv('../DL_model/caseids_age_gender.csv')
    

    ### test set에 해당하는 file_path
    caseid_test = pickle.load(open('../DL_model/caseid_test_new', 'rb'))
    caseid_train = pickle.load(open('../DL_model/caseid_train_new', 'rb'))
    caseid_val = pickle.load(open('../DL_model/caseid_val_new', 'rb'))
    
    non_list = []
    x_train_preop, y_train_preop = [], []
    x_test_preop, y_test_preop = [], []
    x_val_preop,y_val_preop = [], []
    age_train_preop, gender_train_preop = [], []
    age_test_preop, gender_test_preop = [], []
    age_val_preop, gender_val_preop = [], []    

    cnt = 0
    for _, row in df_preprocess_preop.iterrows():
        cnt += 1
        if cnt>1900:
            break
        print('loading data {}/{} ...'.format(cnt, len(df_preprocess_preop)), end='')
        row_demo = df_demograph[df_demograph['caseids']==row['caseids']].iloc[0]
        
        
        # vital data - PPG (resampling 100 Hz to 250 Hz)
        df_vital = pickle.load(open(vital_path+row['file_path'], 'rb')).reset_index()
        pleth_samp = df_vital[['Pleth']].fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        pleth_resamp = signal.resample(pleth_samp, 120*SRATE)
        ppg_per_NRS = np.full(30000, np.nan)
        ppg_per_NRS[0:len(pleth_resamp)] = pleth_resamp
        
        # vital data - ECG (250Hz)       
        try:
            ecg_samp = pickle.load(open(preop_ecg_path+row['file_path'].split(',')[2], 'rb'))
            
        except:
            non_preop_list.append(row['file_path'])
            print('no preop data')
            continue
        else:
            ecg_samp = pd.DataFrame(ecg_samp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()[0:30000]       
            ecg_per_NRS = np.full(30000,np.nan)
            ecg_per_NRS[0:len(ecg_samp)] = ecg_samp
            
            save_path = '../../cranberry2/Preprocessing/cache/lowess_filtered/preop_'+row['caseids']
            # 한 NRS에 대해 23개의 input 확인
            for i in range(0,30):
                # input이 전처리 통과한 경우
                if row[str(i+1)]:
                    start_idx = i*OVERLAP*SRATE # 500i
                    end_idx = (i*OVERLAP+LEN_INPUT)*SRATE # 500i + 1000

                    # input의 normalization
                    ppg_inp = np.copy(ppg_per_NRS[start_idx:end_idx])
                    if np.sum(np.isnan(ppg_inp))!=0:
                        ppg_inp = pd.DataFrame(ppg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()

                    pleth_inp = ppg_inp - smooth(ppg_inp)
                    pleth_inp = pleth_inp - np.nanmean(pleth_inp)

                    ecg_inp = np.copy(ecg_per_NRS[start_idx:end_idx])
                    if np.sum(np.isnan(ecg_inp))!=0:
                        ecg_inp = pd.DataFrame(ecg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()

                    ecg_inp = ecg_inp - smooth(ecg_inp)
                    ecg_inp = (ecg_inp - np.nanmean(ecg_inp)) / np.nanstd(ecg_inp)

                    
                    pickle.dump([pleth_inp, ecg_inp], open(save_path+'_{}'.format(i), 'wb'))
                    # 해당 caseid가 test set에 속하는 경우
                    if row['caseids'] in caseid_test:
                        age_test_preop.append(int(row_demo['age']))
                        if row_demo['gender']=='F':
                            gender_test_preop.append(1)
                        else:
                            gender_test_preop.append(0)
                        x_test_preop.append([pleth_inp, ecg_inp])
                        y_test_preop.append(int(float(row['NRS'])))

                    # 해당 caseid가 train set에 해당하는 경우
                    elif row['caseids'] in caseid_val:
                        age_val_preop.append(int(row_demo['age']))
                        if row_demo['gender']=='F':
                            gender_val_preop.append(1)
                        else:
                            gender_val_preop.append(0)                    
                        x_val_preop.append([pleth_inp, ecg_inp])
                        y_val_preop.append(int(float(row['NRS'])))

                    elif row['caseids'] in caseid_train:
                        age_train_preop.append(int(row_demo['age']))
                        if row_demo['gender']=='F':
                            gender_train_preop.append(1)
                        else:
                            gender_train_preop.append(0)                    
                        x_train_preop.append([pleth_inp, ecg_inp])
                        y_train_preop.append(int(float(row['NRS'])))
                        
                    else:
                        non_list.append(row['caseids'])

            print('completed')

    x_train_preop = np.array(x_train_preop, np.float32)
    x_test_preop = np.array(x_test_preop, np.float32)
    y_train_preop = np.array(y_train_preop, int)
    y_test_preop = np.array(y_test_preop, int)
    x_val_preop = np.array(x_val_preop, np.float32)
    y_val_preop = np.array(y_val_preop, int)

    age_train_preop = np.array(age_train_preop, int)
    age_test_preop = np.array(age_test_preop, int)
    age_val_preop = np.array(age_val_preop, int)
    gender_train_preop = np.array(gender_train_preop, int)
    gender_test_preop = np.array(gender_test_preop, int)
    gender_val_preop = np.array(gender_val_preop, int)


    # 저장하기
    print('saving...', end='', flush=True)
    np.savez_compressed(input_path+'x_train_preop1.npz', x_train_preop)
    np.savez_compressed(input_path+'x_test_preop1.npz', x_test_preop)
    np.savez_compressed(input_path+'x_val_preop1.npz', x_val_preop)
    np.savez_compressed(input_path+'y_train_preop1.npz', y_train_preop)
    np.savez_compressed(input_path+'y_test_preop1.npz', y_test_preop)
    np.savez_compressed(input_path+'y_val_preop1.npz', y_val_preop)

    np.savez_compressed(input_path+'age_train_preop1.npz', age_train_preop)
    np.savez_compressed(input_path+'age_test_preop1.npz', age_test_preop)
    np.savez_compressed(input_path+'age_val_preop1.npz', age_val_preop)    
    np.savez_compressed(input_path+'gender_train_preop1.npz', gender_train_preop)
    np.savez_compressed(input_path+'gender_test_preop1.npz', gender_test_preop)
    np.savez_compressed(input_path+'gender_val_preop1.npz', gender_val_preop)    

    print('done', flush=True)

    
    
    
print('size of training set(preop):', len(x_train_preop))
print('size of validation set(preop):', len(x_val_preop))
print('size of test set(preop):', len(x_test_preop))

loading data 1/3861 ...completed
loading data 2/3861 ...completed
loading data 3/3861 ...completed
loading data 4/3861 ...completed
loading data 5/3861 ...completed
loading data 6/3861 ...completed
loading data 7/3861 ...completed
loading data 8/3861 ...completed
loading data 9/3861 ...completed
loading data 10/3861 ...completed
loading data 11/3861 ...completed
loading data 12/3861 ...completed
loading data 13/3861 ...completed
loading data 14/3861 ...completed
loading data 15/3861 ...completed
loading data 16/3861 ...completed
loading data 17/3861 ...completed
loading data 18/3861 ...completed
loading data 19/3861 ...completed
loading data 20/3861 ...completed
loading data 21/3861 ...completed
loading data 22/3861 ...completed
loading data 23/3861 ...completed
loading data 24/3861 ...completed
loading data 25/3861 ...completed
loading data 26/3861 ...completed
loading data 27/3861 ...completed
loading data 28/3861 ...completed
loading data 29/3861 ...completed
loading data 30/3861 ..

loading data 238/3861 ...completed
loading data 239/3861 ...completed
loading data 240/3861 ...completed
loading data 241/3861 ...completed
loading data 242/3861 ...completed
loading data 243/3861 ...completed
loading data 244/3861 ...completed
loading data 245/3861 ...completed
loading data 246/3861 ...completed
loading data 247/3861 ...completed
loading data 248/3861 ...completed
loading data 249/3861 ...completed
loading data 250/3861 ...completed
loading data 251/3861 ...completed
loading data 252/3861 ...completed
loading data 253/3861 ...completed
loading data 254/3861 ...completed
loading data 255/3861 ...completed
loading data 256/3861 ...completed
loading data 257/3861 ...completed
loading data 258/3861 ...completed
loading data 259/3861 ...completed
loading data 260/3861 ...completed
loading data 261/3861 ...completed
loading data 262/3861 ...completed
loading data 263/3861 ...completed
loading data 264/3861 ...completed
loading data 265/3861 ...completed
loading data 266/386

loading data 473/3861 ...completed
loading data 474/3861 ...completed
loading data 475/3861 ...completed
loading data 476/3861 ...completed
loading data 477/3861 ...completed
loading data 478/3861 ...completed
loading data 479/3861 ...completed
loading data 480/3861 ...completed
loading data 481/3861 ...completed
loading data 482/3861 ...completed
loading data 483/3861 ...completed
loading data 484/3861 ...completed
loading data 485/3861 ...completed
loading data 486/3861 ...completed
loading data 487/3861 ...completed
loading data 488/3861 ...completed
loading data 489/3861 ...completed
loading data 490/3861 ...completed
loading data 491/3861 ...completed
loading data 492/3861 ...completed
loading data 493/3861 ...completed
loading data 494/3861 ...completed
loading data 495/3861 ...completed
loading data 496/3861 ...completed
loading data 497/3861 ...completed
loading data 498/3861 ...completed
loading data 499/3861 ...completed
loading data 500/3861 ...completed
loading data 501/386

loading data 713/3861 ...completed
loading data 714/3861 ...completed
loading data 715/3861 ...completed
loading data 716/3861 ...completed
loading data 717/3861 ...completed
loading data 718/3861 ...completed
loading data 719/3861 ...completed
loading data 720/3861 ...completed
loading data 721/3861 ...completed
loading data 722/3861 ...completed
loading data 723/3861 ...completed
loading data 724/3861 ...completed
loading data 725/3861 ...completed
loading data 726/3861 ...completed
loading data 727/3861 ...completed
loading data 728/3861 ...completed
loading data 729/3861 ...completed
loading data 730/3861 ...completed
loading data 731/3861 ...completed
loading data 732/3861 ...completed
loading data 733/3861 ...completed
loading data 734/3861 ...completed
loading data 735/3861 ...completed
loading data 736/3861 ...completed
loading data 737/3861 ...completed
loading data 738/3861 ...completed
loading data 739/3861 ...completed
loading data 740/3861 ...completed
loading data 741/386

loading data 948/3861 ...completed
loading data 949/3861 ...completed
loading data 950/3861 ...completed
loading data 951/3861 ...completed
loading data 952/3861 ...completed
loading data 953/3861 ...completed
loading data 954/3861 ...completed
loading data 955/3861 ...completed
loading data 956/3861 ...completed
loading data 957/3861 ...completed
loading data 958/3861 ...completed
loading data 959/3861 ...completed
loading data 960/3861 ...completed
loading data 961/3861 ...completed
loading data 962/3861 ...completed
loading data 963/3861 ...completed
loading data 964/3861 ...completed
loading data 965/3861 ...completed
loading data 966/3861 ...completed
loading data 967/3861 ...completed
loading data 968/3861 ...completed
loading data 969/3861 ...completed
loading data 970/3861 ...completed
loading data 971/3861 ...completed
loading data 972/3861 ...completed
loading data 973/3861 ...completed
loading data 974/3861 ...completed
loading data 975/3861 ...completed
loading data 976/386

loading data 1179/3861 ...completed
loading data 1180/3861 ...completed
loading data 1181/3861 ...completed
loading data 1182/3861 ...completed
loading data 1183/3861 ...completed
loading data 1184/3861 ...completed
loading data 1185/3861 ...completed
loading data 1186/3861 ...completed
loading data 1187/3861 ...completed
loading data 1188/3861 ...completed
loading data 1189/3861 ...completed
loading data 1190/3861 ...completed
loading data 1191/3861 ...completed
loading data 1192/3861 ...completed
loading data 1193/3861 ...completed
loading data 1194/3861 ...completed
loading data 1195/3861 ...completed
loading data 1196/3861 ...completed
loading data 1197/3861 ...completed
loading data 1198/3861 ...completed
loading data 1199/3861 ...completed
loading data 1200/3861 ...completed
loading data 1201/3861 ...completed
loading data 1202/3861 ...completed
loading data 1203/3861 ...completed
loading data 1204/3861 ...completed
loading data 1205/3861 ...completed
loading data 1206/3861 ...co

loading data 1406/3861 ...completed
loading data 1407/3861 ...completed
loading data 1408/3861 ...completed
loading data 1409/3861 ...completed
loading data 1410/3861 ...completed
loading data 1411/3861 ...completed
loading data 1412/3861 ...completed
loading data 1413/3861 ...completed
loading data 1414/3861 ...completed
loading data 1415/3861 ...completed
loading data 1416/3861 ...completed
loading data 1417/3861 ...completed
loading data 1418/3861 ...completed
loading data 1419/3861 ...completed
loading data 1420/3861 ...completed
loading data 1421/3861 ...completed
loading data 1422/3861 ...completed
loading data 1423/3861 ...completed
loading data 1424/3861 ...completed
loading data 1425/3861 ...completed
loading data 1426/3861 ...completed
loading data 1427/3861 ...completed
loading data 1428/3861 ...completed
loading data 1429/3861 ...completed
loading data 1430/3861 ...completed
loading data 1431/3861 ...completed
loading data 1432/3861 ...completed
loading data 1433/3861 ...co

loading data 1634/3861 ...completed
loading data 1635/3861 ...completed
loading data 1636/3861 ...completed
loading data 1637/3861 ...completed
loading data 1638/3861 ...completed
loading data 1639/3861 ...completed
loading data 1640/3861 ...completed
loading data 1641/3861 ...completed
loading data 1642/3861 ...completed
loading data 1643/3861 ...completed
loading data 1644/3861 ...completed
loading data 1645/3861 ...completed
loading data 1646/3861 ...completed
loading data 1647/3861 ...completed
loading data 1648/3861 ...completed
loading data 1649/3861 ...completed
loading data 1650/3861 ...completed
loading data 1651/3861 ...completed
loading data 1652/3861 ...completed
loading data 1653/3861 ...completed
loading data 1654/3861 ...completed
loading data 1655/3861 ...completed
loading data 1656/3861 ...completed
loading data 1657/3861 ...completed
loading data 1658/3861 ...completed
loading data 1659/3861 ...completed
loading data 1660/3861 ...completed
loading data 1661/3861 ...co

loading data 1861/3861 ...completed
loading data 1862/3861 ...completed
loading data 1863/3861 ...completed
loading data 1864/3861 ...completed
loading data 1865/3861 ...completed
loading data 1866/3861 ...completed
loading data 1867/3861 ...completed
loading data 1868/3861 ...completed
loading data 1869/3861 ...completed
loading data 1870/3861 ...completed
loading data 1871/3861 ...completed
loading data 1872/3861 ...completed
loading data 1873/3861 ...completed
loading data 1874/3861 ...completed
loading data 1875/3861 ...completed
loading data 1876/3861 ...completed
loading data 1877/3861 ...completed
loading data 1878/3861 ...completed
loading data 1879/3861 ...completed
loading data 1880/3861 ...completed
loading data 1881/3861 ...completed
loading data 1882/3861 ...completed
loading data 1883/3861 ...completed
loading data 1884/3861 ...completed
loading data 1885/3861 ...completed
loading data 1886/3861 ...completed
loading data 1887/3861 ...completed
loading data 1888/3861 ...co